In [74]:
%matplotlib inline
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import skimage.io as io
import pylab
import shutil
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

generate annotation

In [75]:
import os
import json
import pdb
p=pdb.set_trace
def ReadTxt(rootdir):
    lines = []
    with open(rootdir, 'r') as file_to_read:
        while True:
            line = file_to_read.readline()
            if not line:
                break
            line = line.strip('\n')
            lines.append(line)
    return lines


pre_anno=[]
for root, dirs, files in os.walk("./predicted/"):
    for file in files:
        lines=ReadTxt(root+file)
        for line in lines:
            tmp=line.split(' ')[2:]
            bbox=list(map(float,tmp))
            tmp_anno={"image_id":int(file[:-4]),
                      "category_id":1,
                      "bbox":[bbox[0],bbox[1],bbox[2]-bbox[0],bbox[3]-bbox[1]],
                      "score":float(line.split(' ')[1])}
            pre_anno.append(tmp_anno)
with open("pre_eval.json", "w") as outfile:
    json.dump(pre_anno, outfile)

"""
gt_anno=[]
img=[]
id=0
for root, dirs, files in os.walk("../mAP/ground-truth/"):
    for file in files:
        lines=ReadTxt(root+file)
        for line in lines:
            tmp=line.split(' ')[1:]
            tmp_anno={"image_id":int(file[:-4]),
                      "category_id":0,
                      "bbox":list(map(float,tmp)),
                     "id":id,
                     "iscrowd":0}
            
            gt_anno.append(tmp_anno)
            img.append({"id":id})
            id+=1
with open("gt_eval.json", "w") as outfile:
    json.dump({"images":img,
               "annotations":gt_anno,
                "categories": [{"supercategory": "person", "id": 0, "name": "person"}]}, outfile)
"""   

'\ngt_anno=[]\nimg=[]\nid=0\nfor root, dirs, files in os.walk("../mAP/ground-truth/"):\n    for file in files:\n        lines=ReadTxt(root+file)\n        for line in lines:\n            tmp=line.split(\' \')[1:]\n            tmp_anno={"image_id":int(file[:-4]),\n                      "category_id":0,\n                      "bbox":list(map(float,tmp)),\n                     "id":id,\n                     "iscrowd":0}\n            \n            gt_anno.append(tmp_anno)\n            img.append({"id":id})\n            id+=1\nwith open("gt_eval.json", "w") as outfile:\n    json.dump({"images":img,\n               "annotations":gt_anno,\n                "categories": [{"supercategory": "person", "id": 0, "name": "person"}]}, outfile)\n'

evaluate

In [76]:
annType = ['segm','bbox','keypoints']
annType = annType[1]      #specify type here
prefix = 'person_keypoints' if annType=='keypoints' else 'instances'
#print 'Running demo for *%s* results.'%(annType)

In [77]:
#initialize COCO ground truth api
#dataDir='../'
#dataType='val2014'
#annFile = '%s/annotations/%s_%s.json'%(dataDir,prefix,dataType)
annFile=('../data/dataset/instances_val2017.json')
cocoGt=COCO(annFile)

loading annotations into memory...
Done (t=0.41s)
creating index...
index created!


In [78]:
#initialize COCO detections api
#resFile='%s/results/%s_%s_fake%s100_results.json'
#resFile = resFile%(dataDir, prefix, dataType, annType)
resFile=('pre_eval.json')
cocoDt=cocoGt.loadRes(resFile)

Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!


In [79]:
#imgIds=sorted(cocoGt.getImgIds())
#imgIds=imgIds[0:100]
#imgId = imgIds[np.random.randint(100)]
import json 
dts = json.load(open(resFile,'r'))
imgIds = [imid['image_id'] for imid in dts]
imgIds = sorted(list(set(imgIds)))
del dts

In [80]:
# running evaluation
cocoEval = COCOeval(cocoGt,cocoDt,annType)
cocoEval.params.imgIds  = imgIds
cocoEval.params.catIds = [1]
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.90s).
Accumulating evaluation results...
DONE (t=0.24s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.446
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.745
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.461
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.163
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.480
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.680
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.218
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.490
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.500
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.224
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100

In [33]:
# del tmptxt
#shutil.rmtree("./predicted/")

FileNotFoundError: [Errno 2] No such file or directory: './predicted/'